In [2]:
import gmaps, json
from sklearn.decomposition import PCA
import requests, datetime, time, h5py
from sklearn.externals import joblib
import missingno as msno, geopy.distance
from uszipcode import Zipcode, SearchEngine
import geojson
import ipywidgets as widgets 
import geopy

import re, numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
import nltk
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize

%matplotlib inline
pd.options.display.max_columns = 90
pd.options.display.max_rows = 200

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [1]:
features = ['alias', 'name', 'categories', 'coordinates', 'distance', 
            'price', 'rating', 'review_count', 'attributes']

search_radius = 1000  # in meters, 1600m is about 1 mile

main_categories = ['Active Life','Arts & Entertainment','Automotive','Beauty & Spas',
'Education','Event Planning & Services','Financial Services','Food','Health & Medical',
'Home Services','Hotels & Travel','Local Services','Mass Media','Nightlife','Pets',
'Professional Services','Public Services & Government','Real estate','Religious Organizations',
'Restaurants','Shopping']

main_alias = ['active','arts','automotive','beautysvc',
'education','eventservices','financialservices',
'food','health','homeservices','hotelstravel','localservices',
'massmedia','nightlife','pets','professional','publicservicesgovt',
'realestate','religiousorgs','restaurants','shopping']

In [2]:
top_148 = ['Restaurants','Shopping','Food','Beauty & Spas','Home Services','Health & Medical',
'Local Services','Automotive','Nightlife','Bars','Event Planning & Services','Active Life','Fashion',
'Coffee & Tea','Sandwiches','Hair Salons','Fast Food','American (Traditional)','Pizza','Home & Garden',
'Auto Repair','Hotels & Travel','Arts & Entertainment','Professional Services','Doctors','Real Estate',
'Burgers','Breakfast & Brunch','Nail Salons','Fitness & Instruction','Italian','Mexican','Specialty Food',
'Chinese','American (New)','Pets','Hair Removal','Bakeries','Dentists','Grocery','Skin Care','Cafes',
'Education','Desserts','Financial Services','Contractors',"Women's Clothing",'Pet Services',
'General Dentistry','Massage','Ice Cream & Frozen Yogurt','Hotels','Japanese','Chicken Wings',
'Day Spas','Seafood','Salad','Apartments','Waxing','Gyms','Caterers','Tires','Sushi Bars','Flowers & Gifts',
'Venues & Event Spaces','Sports Bars','Oil Change Stations','Accessories','Sporting Goods','Pubs',
'Cosmetics & Beauty Supply','Cosmetic Dentists','Hair Stylists','Barbers','Car Dealers',
'Auto Parts & Supplies','Home Decor','Beer, Wine & Spirits','Trainers','Delis','Furniture Stores',
'Canadian (New)','Asian Fusion','Mediterranean','Convenience Stores','Eyelash Service',"Men's Clothing",
'Barbeque','Massage Therapy','Jewelry','Arts & Crafts','Drugstores','Medical Centers','Department Stores',
'Juice Bars & Smoothies','Steakhouses','Heating & Air Conditioning/HVAC','Electronics','Lounges',
'Pet Groomers','IT Services & Computer Repair','Real Estate Services','Gas Stations','Specialty Schools',
'Indian','Thai','Dry Cleaning','Laundry Services','Shoe Stores','Diners','Chiropractors',
'Books, Mags, Music & Video','Plumbing','Party & Event Planning','Public Services & Government',
'Banks & Credit Unions','Veterinarians','Pet Sitting','Optometrists','Vietnamese','Real Estate Agents',
'Body Shops','Home Cleaning','Makeup Artists','Mobile Phones','Middle Eastern','Medical Spas','Yoga',
'Landscaping','Pet Stores','Cocktail Bars','Car Wash','Wine Bars','Eyewear & Opticians','Printing Services',
'Florists','Movers','Auto Detailing','Greek','Vegetarian','Photographers','Transportation',
'Performing Arts','Hair Extensions','Lawyers','French','Self Storage','Physical Therapy']

148

In [ ]:
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0

In [2]:
def calcDist(lat1, long1, lat2, long2) :  # returns distance in meters
    try : d = geopy.distance.distance((lat1,long1), (lat2,long2)).m
    except : d = np.nan
    return d

In [ ]:
def calcDistances(lat, long, citydf) :
    distances = pd.Series(index=citydf.index)
    for index in citydf.index :
        distances[index] = calcDist(lat, long, citydf.loc[index,'latitude'], citydf.loc[index,'longitude'])
    return distances

In [ ]:
def latlong_tozipcode(tupl) :
    try :
        (lat, long) = tupl
        result = search.by_coordinates(lat, long, radius=10, returns=1)
        int(result[0].to_dict()['zipcode'])
    except : return 0
    return int(result[0].to_dict()['zipcode'])